In [1]:
#restart till get t4 or p4
!nvidia-smi

Sun May  2 13:37:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install wandb
# import wandb
# wandb.init()

In [3]:
# #get 25 gb if necessary
# https://towardsdatascience.com/double-your-google-colab-ram-in-10-seconds-using-these-10-characters-efa636e646ff

In [4]:
# sources
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=G-kkz81OY6xH
#https://www.kaggle.com/samarthagarwal23/bert-lm-fine-tune-using-gpu-on-wikitext
#https://modelzoo.co/model/pytorch-pretrained-bert#Fine-tuning-with-BERT-running-the-examples
#https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=LTXXutqeDzPi


In [5]:
pip install transformers

     |████████████████████████████████| 2.1MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 36.8MB/s 
     |████████████████████████████████| 901kB 55.1MB/s 


In [6]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

#a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc")

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transfer learning

In [8]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32001, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [10]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [11]:
model.num_parameters()
#110 millio!!

110651649

Prepare input

In [ ]:
# !wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    # file_path="./oscar.eo.txt",
    file_path="./corpustext_openrefined_and_normalunct_v20.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 1min 9s, sys: 648 ms, total: 1min 10s
Wall time: 1min 10s


In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Myfinetuded",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [17]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,1.484700
1000,0.857800
1500,0.827000
2000,0.814800
2500,0.799300
3000,0.792800
3500,0.805200
4000,0.773100
4500,0.781900
5000,0.776600


CPU times: user 5h 43min 11s, sys: 2h 50min 45s, total: 8h 33min 57s
Wall time: 4h 42min 48s


TrainOutput(global_step=59420, training_loss=0.7419580342110782, metrics={'train_runtime': 16967.7174, 'train_samples_per_second': 3.502, 'total_flos': 2.152456617464202e+17, 'epoch': 20.0, 'init_mem_cpu_alloc_delta': 1862864896, 'init_mem_gpu_alloc_delta': 443394560, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 15478784, 'train_mem_gpu_alloc_delta': 7536128, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 4396078592})

save the model

In [18]:
#save model
trainer.save_model("./drive/MyDrive/berthunv11_mese_20epoch_64batch_mondat")

In [19]:
#save model
trainer.save_model("./Myfinetuded")

In [20]:
#save model
trainer.save_model("./drive/MyDrive/berthunv11_mese_20epoch_64batch_mondat")

check if mask does work

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./Myfinetuded",
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
)

In [ ]:
#test
fill_mask("A tengeri hajózás" + tokenizer.mask_token +".")

[{'score': 0.8575894236564636,
  'sequence': 'A tengeri hajózás.',
  'token': 0,
  'token_str': '[ P A D ]'},
 {'score': 0.09243928641080856,
  'sequence': 'A tengeri hajózás szabályos.',
  'token': 23474,
  'token_str': 's z a b á l y o s'},
 {'score': 0.012780212797224522,
  'sequence': 'A tengeri hajózás biztonságos.',
  'token': 7928,
  'token_str': 'b i z t o n s á g o s'},
 {'score': 0.006540424190461636,
  'sequence': 'A tengeri hajózás normális.',
  'token': 9172,
  'token_str': 'n o r m á l i s'},
 {'score': 0.00271251960657537,
  'sequence': 'A tengeri hajózás szélesebb.',
  'token': 12649,
  'token_str': 's z é l e s e b b'}]